In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"



Using TensorFlow backend.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd



path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/MSR-Cambridge/Part2" 

names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']


#['TimeStamp','Response','IOType','LUN','ByteOffset','Size']


all_files = glob.glob(os.path.join(path, "usr_1.csv"))
print(all_files)




Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


KeyboardInterrupt: 

In [4]:
df = pd.read_csv(all_files[0],engine='python',skiprows =2,header=None,na_values=['-1'], index_col=False) 
names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']
df.columns = names
print (len(df))

1211033


In [5]:
from collections import Counter
print(len(Counter(df['IOSize'])))
print(len(Counter(df['ByteOffset'])))

143
115582


In [6]:

df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
df = df.drop(df.index[-1])




In [7]:
df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

a = df['ByteOffset_Delta'].unique().tolist()
operation_id_map = {}
for i,id in enumerate(a): operation_id_map[id] = i 
df['ByteOffset_Delta_class'] = df['ByteOffset_Delta'].map(lambda x: operation_id_map[x])


In [8]:
a = df['IOSize'].unique().tolist()
size_id_map = {}
for i,id in enumerate(a): size_id_map[id] = i 
df['Size_class'] = df['IOSize'].map(lambda x: size_id_map[x])

In [9]:
#Sorting df by TimeStamp

df = df.sort_values(by=['TimeStamp'])
df.reset_index(inplace=True, drop=True)
print(df.dtypes)
print(len(df))

TimeStamp                   int64
Host_Name                  object
DiskNumber                float64
Operation_Type             object
ByteOffset                  int64
IOSize                      int64
Response_Time               int64
DiskNum                     int64
ByteOffset_Delta          float64
ByteOffset_Delta_class      int64
Size_class                  int64
dtype: object
1211032


In [10]:
df.head(5)

,TimeStamp,Host_Name,DiskNumber,Operation_Type,ByteOffset,IOSize,Response_Time,DiskNum,ByteOffset_Delta,ByteOffset_Delta_class,Size_class
0,128166372011594132,mds,NaN,Write,3154132992,4096,51785,0,-4.752998e+07,0,0
1,128166372011600556,mds,NaN,Write,3201662976,20480,45361,0,4.753818e+07,1,1
2,128166372011606862,mds,NaN,Write,3154124800,4096,39055,0,3.096306e+09,2,0
3,128166372017062367,mds,NaN,Write,57819136,4096,52301,0,-4.096000e+03,3,0
4,128166372017074528,mds,NaN,Write,57823232,4096,40140,0,1.638400e+04,4,0


In [11]:
from collections import Counter
x = Counter(df['ByteOffset_Delta_class'])
vals = {}
vals =  x.most_common(1000)
bo_list = []

for x in vals:
    bo_list.append(x[0])
        
count = 0
label_list = []

while (count < len(df)):
    x = df['ByteOffset_Delta_class'].iloc[count]
    if x in bo_list:
        label_list.append(x)
    else:
        label_list.append(999999)
    count= count + 1
    
ByteOffset_Delta_class_backup  = df['ByteOffset_Delta_class'] 
df['ByteOffset_Delta_class']  = label_list
print(len(Counter(df['ByteOffset_Delta_class'])))
    
a = df['ByteOffset_Delta_class'].unique().tolist()
bo_map = {}
for i,id in enumerate(a): bo_map[id] = i 
df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta_class'].map(lambda x: bo_map[x])
    
label_list = df['ByteOffset_Delta_Class_1001'] 
    
df['ByteOffset_Delta_Class_1001']  = label_list
    

1001


In [12]:
len(Counter(df['ByteOffset_Delta_Class_1001']))

1001

In [13]:
len(Counter(df['Size_class']))

143

In [14]:
df.dtypes

TimeStamp                        int64
Host_Name                       object
DiskNumber                     float64
Operation_Type                  object
ByteOffset                       int64
IOSize                           int64
Response_Time                    int64
DiskNum                          int64
ByteOffset_Delta               float64
ByteOffset_Delta_class           int64
Size_class                       int64
ByteOffset_Delta_Class_1001      int64
dtype: object

In [15]:
df.head(5)

,TimeStamp,Host_Name,DiskNumber,Operation_Type,ByteOffset,IOSize,Response_Time,DiskNum,ByteOffset_Delta,ByteOffset_Delta_class,Size_class,ByteOffset_Delta_Class_1001
0,128166372011594132,mds,NaN,Write,3154132992,4096,51785,0,-4.752998e+07,999999,0,0
1,128166372011600556,mds,NaN,Write,3201662976,20480,45361,0,4.753818e+07,999999,1,0
2,128166372011606862,mds,NaN,Write,3154124800,4096,39055,0,3.096306e+09,999999,0,0
3,128166372017062367,mds,NaN,Write,57819136,4096,52301,0,-4.096000e+03,3,0,1
4,128166372017074528,mds,NaN,Write,57823232,4096,40140,0,1.638400e+04,4,0,2


In [16]:
# Drop unnecessary coloumns

print("Before drop: {}".format(df.columns))
df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]], axis=1,inplace=True) 


Before drop: Index(['TimeStamp', 'Host_Name', 'DiskNumber', 'Operation_Type', 'ByteOffset',
       'IOSize', 'Response_Time', 'DiskNum', 'ByteOffset_Delta',
       'ByteOffset_Delta_class', 'Size_class', 'ByteOffset_Delta_Class_1001'],
      dtype='object')


In [17]:
df.dtypes

Size_class                     int64
ByteOffset_Delta_Class_1001    int64
dtype: object

In [18]:
from collections import Counter
print(len(Counter(df['Size_class'])))

143


In [19]:
from collections import Counter
print(len(Counter(df['ByteOffset_Delta_Class_1001'])))

1001


In [20]:
df.head(5)

,Size_class,ByteOffset_Delta_Class_1001
0,0,0
1,1,0
2,0,0
3,0,1
4,0,2


In [21]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size_class'].tolist()
size_test = df[training_pt_1+1:]['Size_class'].tolist()


In [22]:
data_path  = r'/soe/cchakrab/test_output/output_csv/SYSTOR/p3/'

path_train  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/lba_train.txt"

with open(path_train, 'w') as f:
    for item in lba_train:
        f.write("%s " % item)
        
path_test  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/lba_test.txt"
with open(path_test, 'w') as f:
    for item in lba_test:
        f.write("%s " % item)

        
path_train  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/size_train.txt"

with open(path_train, 'w') as f:
    for item in size_train:
        f.write("%s " % item)
        
path_test  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/size_test.txt"
with open(path_test, 'w') as f:
    for item in size_test:
        f.write("%s " % item)

In [23]:
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import argparse




# parser = argparse.ArgumentParser()
# parser.add_argument('run_opt', type=int, default=1, help='An integer: 1 to train, 2 to test')
# parser.add_argument('--data_path', type=str, default=data_path, help='The full path of the training data')
# args = parser.parse_args()


# if args.data_path:
#     data_path = args.data_path


# def read_words(filename):
#     with tf.gfile.GFile(filename, "r") as f:
#         return f.read().split()

def read_words(filename):
    with open(filename, 'r') as f:
        return f.read().split()

def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]



def load_data_lba():
    # get the data paths
    train_path = os.path.join(data_path, "lba_train.txt")
    test_path = os.path.join(data_path, "lba_test.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    #print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, test_data, vocabulary, reversed_dictionary




def load_data_size():
    # get the data paths
    train_path = os.path.join(data_path, "size_train.txt")
    test_path = os.path.join(data_path, "size_test.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    #print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return np.array(train_data), np.array(test_data), vocabulary, reversed_dictionary



In [24]:
train_data_lba, test_data_lba, vocabulary_1, reversed_dictionary_1 = load_data_lba()
train_data_size, test_data_size, vocabulary_2, reversed_dictionary_2 = load_data_size()

[0, 0, 0, 3, 70]
986
0 0 0 1 2 0 0 0 0 0
[0, 14, 0, 0, 0]
141
0 1 0 0 0 0 0 0 0 0


In [33]:
# Two classification outputs
import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf






from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate , Dot
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Reshape




# no_docs = len(y_train_lba)
maxlen= 32



# # define two sets of inputs
# inputA = Input(shape=(32,))
# inputB = Input(shape=(32,))
# # inputA = Sequential()
# # inputB = Sequential()
vocabulary_1 = 1001
vocabulary_2 = 143
hidden_size = 1500

# input=Input(shape=(no_docs,maxlen),dtype='float64')
inputA=Input(shape=(maxlen,),dtype='float64')  
inputB=Input(shape=(maxlen,),dtype='float64') 


# the first branch operates on the first input
x = Embedding(input_dim=vocabulary_1,output_dim=250,input_length=maxlen)(inputA)
x = Model(inputs=inputA, outputs=x)

# # the second branch opreates on the second input
y = Embedding(input_dim=vocabulary_2,output_dim=250,input_length=maxlen)(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = keras.layers.concatenate([x.output, y.output])

lstm1 = LSTM(hidden_size,return_sequences=True)(combined)
lstm2 = LSTM(hidden_size, return_sequences=True)(lstm1)

# create classification output
offset = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_1, activation='softmax'), name='offset')(lstm2)
iosize = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_2, activation='softmax'), name='iosize')(lstm2)




model =Model([inputA,inputB],[offset,iosize]) # combining all into a Keras model

model.compile(optimizer='rmsprop',
              loss={'offset': 'categorical_crossentropy', 'iosize': 'categorical_crossentropy'},
              loss_weights={'offset': 2., 'iosize': 1.5},
              metrics={ 'offset': 'categorical_accuracy', 'iosize': 'categorical_accuracy'})
model.summary()


Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 32, 250)      250250      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 32, 250)      35750       input_6[0][0]                    
____________________________________________________________________________________________

In [34]:
class KerasBatchGenerator(object):

    def __init__(self, data_1, data_2, num_steps, batch_size, vocabulary1,vocabulary2, skip_step=0):
        self.data_1 = data_1
        self.data_2 = data_2
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary1 = vocabulary1
        self.vocabulary2 = vocabulary2
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x_lba = np.zeros((self.batch_size, self.num_steps))
        x_size = np.zeros((self.batch_size, self.num_steps))
        y_lba = np.zeros((self.batch_size, self.num_steps, self.vocabulary1))
        y_size = np.zeros((self.batch_size, self.num_steps, self.vocabulary2))
        while True:
            for i in range(self.batch_size):
                predict_ahead = 32
                if self.current_idx + self.num_steps >= len(self.data_1):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x_lba[i, :] = self.data_1[self.current_idx:self.current_idx + self.num_steps]
                x_size[i, :] = self.data_2[self.current_idx:self.current_idx + self.num_steps]
                
                temp_y_lba = self.data_1[self.current_idx + predict_ahead:self.current_idx + self.num_steps + 1]
                temp_y_size = self.data_2[self.current_idx + predict_ahead:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y_lba[i, :, :] = to_categorical(temp_y_lba, num_classes=self.vocabulary1)
                y_size[i, :, :] = to_categorical(temp_y_size, num_classes=self.vocabulary2)
                self.current_idx += self.skip_step
                
            #yield (np.array(x_lba),np.array(x_size)), (np.array(y_lba),np.array(y_size))
            yield [x_lba, x_size],[y_lba, y_size]
#            yield [np.array(top_batch), np.array(bot_batch)], np.array(batch_labels)
#             print(x)
#             print(y)
#             yield x,y


In [35]:
num_steps = 32
batch_size = 32

train_data_generator = KerasBatchGenerator(train_data_lba,train_data_size, num_steps, batch_size, vocabulary_1,vocabulary_2,skip_step=0) 
test_data_generator = KerasBatchGenerator(test_data_lba,test_data_size, num_steps, batch_size, vocabulary_1, vocabulary_2, skip_step=0)


In [39]:
import time
num_epochs = 1000
batch_size = 32
num_steps = 32


monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=2, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath=data_path + 'best_weights-4.hdf5', verbose=1,save_best_only=True)

print('Train...')
start_time = time.time()

# model.fit([X_train_lba,X_train_size],[y_train_lba,y_train_size],
#           verbose=1,epochs=1000,callbacks=[monitor,checkpointer])

valid_steps = len(train_data_lba)//(batch_size* 32)
train_steps = len(train_data_lba)//(batch_size *32)

model.fit_generator(train_data_generator.generate(),
                    train_steps,
                    num_epochs,
                    verbose=1,
                    validation_data=test_data_generator.generate(),
                    validation_steps= valid_steps,
                    callbacks=[checkpointer,monitor])


# model.fit_generator([train_data_generator_lba.generate(),train_data_generator_size.generate()],
#                     train_steps,
#                     num_epochs,
#                     verbose=1,
#                     validation_data=[test_data_generator_lba.generate(),test_data_generator_size],
#                     validation_steps= valid_steps)

end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))




Train...
Epoch 1/1000
886/886 [==============================] - 183s 207ms/step - loss: 4.1723e-07 - offset_loss: 1.1921e-07 - iosize_loss: 1.1921e-07 - offset_categorical_accuracy: 1.0000 - iosize_categorical_accuracy: 1.0000 - val_loss: 55.7064 - val_offset_loss: 15.9245 - val_iosize_loss: 15.9052 - val_offset_categorical_accuracy: 0.0000e+00 - val_iosize_categorical_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 55.70641, saving model to /soe/cchakrab/test_output/output_csv/SYSTOR/p3/best_weights-4.hdf5
Epoch 2/1000
886/886 [==============================] - 184s 207ms/step - loss: 4.1723e-07 - offset_loss: 1.1921e-07 - iosize_loss: 1.1921e-07 - offset_categorical_accuracy: 1.0000 - iosize_categorical_accuracy: 1.0000 - val_loss: 55.7064 - val_offset_loss: 15.9245 - val_iosize_loss: 15.9052 - val_offset_categorical_accuracy: 0.0000e+00 - val_iosize_categorical_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 55.70641
Epoch 3/1000
231/886 [======>..

KeyboardInterrupt: 

In [ ]:
import time
start_time = time.time()
batch_size = 32
samples = (len(size_test)/batch_size)
score = model.evaluate_generator(test_data_generator.generate(), samples, verbose = 1)
print("Loss: ", score[0], "Accuracy: ", score[1])
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# nb_validation_samples = len(size_test)
# pred_1,pred_2 = model.predict_generator(test_data_generator.generate(), nb_validation_samples // batch_size + 1, verbose = 1)
# # y_pred_1 = np.argmax(pred_1, axis = 1)
# # y_pred_2 = np.argmax(pred_2, axis = 1)

In [ ]:
# from sklearn.metrics import precision_recall_fscore_support
# precision_recall_fscore_support(test_data_lba[32:], y_pred_1, average='macro')

In [ ]:
# from sklearn.metrics import precision_recall_fscore_support
# precision_recall_fscore_support(test_data_size[32:], y_pred_2, average='macro')

In [ ]:
# from sklearn.metrics import accuracy_score
# max_lba_accuracy = 0
# max_lba_accuracy_pos = 0

# max_size_accuracy = 0
# max_size_accuracy_pos = 0

# import time
# start_time = time.time()

# for i in range(32):
#     pred_1 = pred1[:,i,:]
#     pred_2 = pred2[:,i,:]
#     pred_1 = np.argmax(pred_1, axis=1)
#     pred_2 = np.argmax(pred_2, axis=1)
    
    
#     tmp_lba = accuracy_score(lba_test_final, pred_1)
#     if (tmp_lba > max_lba_accuracy):
#         max_lba_accuracy = tmp_lba
#         max_lba_accuracy_pos = i
        
#     tmp_size = accuracy_score(lba_size_final, pred_2)
#     if (tmp_size > max_size_accuracy):
#         max_size_accuracy = tmp_size
#         max_size_accuracy_pos = i
    

# print("Best IO Size Accuracy", str(max_size_accuracy))
# print("Best IO Size Pos", str(max_size_accuracy_pos))

# print("Best LBA Delta Accuracy", str(max_lba_accuracy))
# print("Best LBA Delta Position", str(max_lba_accuracy_pos))

# print("Elapsed time: {}".format((elapsed_time)))

In [ ]:
print("Done...!!")